In [1]:
import os, re, glob, json
from pathlib import Path
import numpy as np
import pandas as pd
import nibabel as nib
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm.second_level import SecondLevelModel, non_parametric_inference
from nilearn import image
import matplotlib.pyplot as plt
from nilearn.plotting import plot_design_matrix
from nilearn.plotting import plot_stat_map
from nilearn.glm import threshold_stats_img
from nilearn import plotting
from nilearn.image import load_img, resample_to_img
from sklearn.model_selection import LeaveOneGroupOut
from nilearn.decoding import Decoder
from sklearn.metrics import classification_report


In [2]:
BASE = "/local/anpa/ds003242-1.0.0"
DERIVATIVES = f"{BASE}/derivatives/"
DERIVATIVES_FMRIPREP = f"{DERIVATIVES}/fmriprep/" 
FIRST_LEVEL_SEP_RUNS = f"{DERIVATIVES}/firstlevel_separate_runs/"  
TASK = "CIC"
TR = 2.0

In [3]:
def resample_mask_to_bold(anat_mask, bold_img):
    """Resample anat mask to the space of the bold image.
    As in tutorial https://nilearn.github.io/dev/auto_examples/04_glm_first_level/plot_first_level_details.html
    """
    return resample_to_img(
        anat_mask,
        bold_img,
        interpolation="nearest",
        copy_header=True,
        force_resample=True,
    )

def fprep_func_dir(sub):
    return Path(BASE)/f"derivatives/fmriprep/sub-{sub}/func"

In [4]:
all_sub_dirs = sorted([p.name.split("sub-")[-1] for p in Path(FIRST_LEVEL_SEP_RUNS).glob("sub-*") if p.is_dir()])
all_sub_dirs[:3]

['SAXSISO01b', 'SAXSISO01f', 'SAXSISO01s']

In [5]:
fasting_participants = [s for s in all_sub_dirs if s.endswith("f")]
social_participants = [s for s in all_sub_dirs if s.endswith("s")]
baseline_participants = [s for s in all_sub_dirs if s.endswith("b")]

In [6]:
fasting_food = [
    f
    for s in fasting_participants
    for f in Path(f"{FIRST_LEVEL_SEP_RUNS}/sub-{s}").rglob("*Food_*_zmap.nii.gz")]
fasting_social = [
    f
    for s in fasting_participants
    for f in Path(f'{FIRST_LEVEL_SEP_RUNS}/sub-{s}').rglob("*Social_*_zmap.nii.gz")]
fasting_control = [
    f
    for s in fasting_participants
    for f in Path(f'{FIRST_LEVEL_SEP_RUNS}/sub-{s}').rglob("*Control_*_zmap.nii.gz")]


social_food = [
    f
    for s in social_participants
    for f in Path(f'{FIRST_LEVEL_SEP_RUNS}/sub-{s}').rglob("*Food_*_zmap.nii.gz")]
social_social = [
    f
    for s in social_participants
    for f in Path(f'{FIRST_LEVEL_SEP_RUNS}/sub-{s}').rglob("*Social_*_zmap.nii.gz")]
social_control = [
    f
    for s in social_participants
    for f in Path(f'{FIRST_LEVEL_SEP_RUNS}/sub-{s}').rglob("*Control_*_zmap.nii.gz")]


baseline_food = [
    f
    for s in baseline_participants
    for f in Path(f'{FIRST_LEVEL_SEP_RUNS}/sub-{s}').rglob("*Food_*_zmap.nii.gz")]
baseline_social = [
    f
    for s in baseline_participants
    for f in Path(f'{FIRST_LEVEL_SEP_RUNS}/sub-{s}').rglob("*Social_*_zmap.nii.gz")]
baseline_control = [
    f
    for s in baseline_participants
    for f in Path(f'{FIRST_LEVEL_SEP_RUNS}/sub-{s}').rglob("*Control_*_zmap.nii.gz")]

In [21]:
def compare(labels: tuple, z_maps: tuple):

    runs_1 = [str(f).split('/')[-1][0] for f in z_maps[0]]
    labels_1 = [labels[0]] * len(z_maps[0])
    runs_2 = [str(f).split('/')[-1][0] for f in z_maps[1]]
    labels_2 = [labels[1]] * len(z_maps[1])

    X = np.array(z_maps[0] + z_maps[1])
    y = np.array(labels_1 + labels_2)
    groups = np.array(runs_1 + runs_2)

    decoder = Decoder(t_r=TR, estimator='svc', scoring='accuracy', mask=None, standardize=True, cv=LeaveOneGroupOut(), n_jobs=-1)
    decoder.fit(X, y, groups=groups)

    classification_accuracy = np.mean(list(decoder.cv_scores_.values()))
    print(
        f"Classification accuracy: {classification_accuracy:.4f} / "
    )

    for label, scores in decoder.cv_scores_.items():
        print(label, np.mean(scores))

    return decoder, (X, y)

# Fasting day VS Baseline day. Binary classification. Food VS Control

Cross classification accuracy should be decent here. Same tasks, different conditions

In [23]:
decoder_ff_fc, data_ff_fc = compare(('Food', 'Control'), (fasting_food, fasting_control))

Classification accuracy: 0.8497 / 
Control 0.8496863799283153
Food 0.8496863799283153


In [24]:
decoder_bf_bc, data_bf_bc = compare(('Food', 'Control'), (baseline_food, baseline_control))

Classification accuracy: 0.8001 / 
Control 0.80009520609319
Food 0.80009520609319


In [25]:
# Lets check cross classification

y_pred = decoder_ff_fc.predict(data_bf_bc[0])
y_true = data_bf_bc[1]
print(classification_report(y_true, y_pred, target_names=['Control', 'Food']))

              precision    recall  f1-score   support

     Control       0.78      0.82      0.80       573
        Food       0.81      0.77      0.79       573

    accuracy                           0.79      1146
   macro avg       0.80      0.79      0.79      1146
weighted avg       0.80      0.79      0.79      1146



In [26]:
y_pred = decoder_bf_bc.predict(data_ff_fc[0])
y_true = data_ff_fc[1]
print(classification_report(y_true, y_pred, target_names=['Control', 'Food']))

              precision    recall  f1-score   support

     Control       0.81      0.86      0.83       570
        Food       0.85      0.80      0.82       570

    accuracy                           0.83      1140
   macro avg       0.83      0.83      0.83      1140
weighted avg       0.83      0.83      0.83      1140



# Social isolation day VS Baseline day. Binary classification. Social VS Control

Cross classification accuracy should be decent here. Same tasks, different conditions

In [35]:
decoder_ss_sc, data_ss_sc = compare(('Social', 'Control'), (social_social, social_control))

Classification accuracy: 0.9566 / 
Control 0.9565972222222223
Social 0.9565972222222223


In [30]:
decoder_bs_bc, data_bs_bc = compare(('Social', 'Control'), (baseline_social, baseline_control))

Classification accuracy: 0.9353 / 
Control 0.9352598566308243
Social 0.9352598566308243


In [36]:
# Lets check cross classification

y_pred = decoder_ss_sc.predict(data_bs_bc[0])
y_true = data_bs_bc[1]
print(classification_report(y_true, y_pred, target_names=['Control', 'Social']))

              precision    recall  f1-score   support

     Control       0.91      0.99      0.95       573
      Social       0.98      0.90      0.94       573

    accuracy                           0.94      1146
   macro avg       0.95      0.94      0.94      1146
weighted avg       0.95      0.94      0.94      1146



In [37]:
y_pred = decoder_bs_bc.predict(data_ss_sc[0])
y_true = data_ss_sc[1]
print(classification_report(y_true, y_pred, target_names=['Control', 'Social']))

              precision    recall  f1-score   support

     Control       0.94      0.99      0.96       576
      Social       0.99      0.93      0.96       576

    accuracy                           0.96      1152
   macro avg       0.96      0.96      0.96      1152
weighted avg       0.96      0.96      0.96      1152



# Cross classification. Food vs Control on fasting day VS Social pictures vs Control on social isolation day

In [38]:
y_pred = decoder_ff_fc.predict(data_ss_sc[0])
y_pred[y_pred == 'Food'] = "Social"

y_true = data_ss_sc[1]

print(classification_report(y_true, y_pred, target_names=['Control', 'Social']))

              precision    recall  f1-score   support

     Control       0.46      0.48      0.47       576
      Social       0.46      0.44      0.45       576

    accuracy                           0.46      1152
   macro avg       0.46      0.46      0.46      1152
weighted avg       0.46      0.46      0.46      1152



In [39]:
y_pred = decoder_ss_sc.predict(data_ff_fc[0])
y_pred[y_pred == 'Social'] = "Food"

y_true = data_ff_fc[1]

print(classification_report(y_true, y_pred, target_names=['Control', 'Food']))

              precision    recall  f1-score   support

     Control       0.45      0.45      0.45       570
        Food       0.45      0.46      0.46       570

    accuracy                           0.45      1140
   macro avg       0.45      0.45      0.45      1140
weighted avg       0.45      0.45      0.45      1140

